# Profitable App Profiles for the App Store and Google Play Market

## Opening and Exploring the Data

As of September 2018, there were approximately 2 million IOS apps available on the App Store and 2.1 million Android apps on Google Play

Source: Statista Collecting data for over four million apps requires a significant amount of time and money, I we'll try to analyze a sample of data instead. To avoid spending resources with collecting new data ourselves, we should first try to see whether we can find any relevant existing data at no cost. Luckily, these are two data sets that seem suitable for our purpose:

A data set containing data about approximately ten thousand Android apps from Google Play. You can download the data set directly from this link. A data set containing data about approximately seven thousand iOS apps from the App Store. You can download the data set directly from this link. Let's start by opening the two data sets and then continue with exploring the data.

In [1]:
import pandas as pd
import numpy as np


In [2]:
from csv import reader

### The Google Play data set ###
with open('googleplaystore.csv', encoding='utf-8') as opened_file:
    read_file = reader(opened_file)
    android = list(read_file)
    android_header = android[0]
    android = android[1:]

### The Appstore data set ###
with open('AppleStore.csv', encoding = 'utf-8') as opened_file:
    read_file = reader(opened_file)
    ios = list(read_file)
    ios_header = ios[0]
    ios = ios[1:]

To make it easier to explore the two data sets, I frist wrote a function named explore_data() that we can use repeatedly to explore rows and columns for any data set.

In [3]:
def explore_data(dataset, start, end, rows_and_columns=False):
    dataset_slice = dataset[start:end]
    for row in dataset_slice:
        print(row)
        print('\n')
    if rows_and_columns:
        print('Number of rows: ', len(dataset))
        print('Number of columns: ', len(dataset[0]))
    print(android_header)
    print('\n')
    explore_data(android, 0, 3, True)

## Deleting Wrong Data

The Google Play data set has an error for row 10472. Let's print this row and compare it against the header and another row that is correct.

In [4]:
print(android[10472]) #wrong row
print('\n')
print(android_header)
print('\n')
print(android[0]) #correct row

['Life Made WI-Fi Touchscreen Photo Frame', '1.9', '19', '3', '1,000+', 'Free', '0', 'Everyone', '', '11-Feb-18', '1.0.19', '4.0 and up', '']


['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', '7-Jan-18', '1.0.0', '4.0.3 and up']


In [5]:
print(len(android))
del android[10472]
print(len(android))

10841
10840


## Removing Duplicate Entries

### Part One 

If we explore the Google Paly data set long enough, we'll find that some apps have more than one entry. For instance, the application Instagram has four entries:

In [6]:
for app in android:
    name = app[0]
    if name == 'Instagram':
        print(app)
        

['Instagram', 'SOCIAL', '4.5', '66577313', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', '31-Jul-18', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66577446', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', '31-Jul-18', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66577313', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', '31-Jul-18', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66509917', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', '31-Jul-18', 'Varies with device', 'Varies with device']


In total, there are 1,181 cases where an app occurs more than once on Android:

In [7]:
duplicate_apps = []
unique_apps = []

for app in android:
    name = app[0]
    if name in unique_apps:
        duplicate_apps.append(name)
    else:
        unique_apps.append(name)
print('Number of duplicate apps: ', len(duplicate_apps))
print('\n')
print('Number of uniquew apps: ' , len(unique_apps))

Number of duplicate apps:  1181


Number of uniquew apps:  9659


In [8]:
duplicate_apps = []
unique_apps = []

for app in ios:
    name = app[0]
    if name in unique_apps:
        duplicate_apps.append(name)
    else:
        unique_apps.append(name)
        
print('Number of duplicate apps: ', len(duplicate_apps))
print('\n')
print('Number of uniquew apps: ' , len(unique_apps))

Number of duplicate apps:  0


Number of uniquew apps:  7197


### Part two 

To remove the duplicates, I will do the following:

- Create a dictionary, where each dictionary key is unique app name and the corresponding dictionary value is the highest number of reviews of that apps.
- Use the information stored in the dictionary and create a new dataset, which will have only one entry per app (and for each app, I'll only select the entry with the highest number of reviews)

In [9]:
reviews_max = {}

for app in android:
    name = app[0]
    n_review = float(app[3])
    
    if name in reviews_max and n_review > reviews_max[name]:
        reviews_max[name] = n_review
    elif name not in reviews_max:
        reviews_max[name] = n_review

In the previous code cell, I found that there are 1,181 cases where an app occurs more than once, so the length of our dictionary (of unique apps) should be equal to the difference between the length of our data and 1,181.

In [10]:
print('Expected length: ', len(android[1:]) - 1181)
print('Actual length: ', len(reviews_max))

Expected length:  9658
Actual length:  9659


In [11]:
android_clean = []
already_added = []

for app in android:
    name = app[0]
    n_review = float(app[3])
    
    if (reviews_max[name] == n_review) and (name not in already_added):
        android_clean.append(app)
        already_added.append(app)


Now let's quickly explore the new data set, and confirm that the number of rows is 9,660.

## Removing Non-English Apps

### Part One

Remember we use English for the apps we develop at our company, and I'd like to analyze only the apps that are designed for an English-speking audience. However, if I explore the data long enough, I'll find that both datasets have apps with names that suggest they are not designed for an English-speadking audience.

We're not interested in keeping these apps, so we'll remove them. One way to do this is to remove each app with a name containing a symbol that isn't commonly used in English text — English text usually includes letters from the English alphabet, numbers composed of digits from 0 to 9, punctuation marks (., !, ?, ;), and other symbols (+, *, /).

In [12]:
print(ios[813][1])
print(ios[6731][1])

print(android_clean[4412][0])
print(android_clean[7940][0])

爱奇艺PPS -《欢乐颂2》电视剧热播
【脱出ゲーム】絶対に最後までプレイしないで 〜謎解き＆ブロックパズル〜
Freeland AH
SONIC Drive-In


All these characters that are specific to English texts are encoded using the ASCII standard. Each ASCII character has a corresponding number between 0 and 127 associated with it, and we can take advantage of that to build a function that checks an app name and tells us whether it contains non-ASCII characters.
I built this function below, and we use the built-in  ord() function to find out the corresponding excoding number of each character.

In [13]:
def is_english(string):
    
    for character in string:
        if ord(character) > 127:
            return False
        return True
print(is_english('Instagram'))
print(is_english('爱奇艺PPS -《欢乐颂2》电视剧热播'))

True
False


The function seems to work fine, but some English app names use emojis or other symbols that fall outside of the ASCII range. Because of this, we'll remove usefull apps if we use the function in its current form.

In [14]:
print(is_english('Docs To Go™ Free Office Suite'))
print(is_english('Instachat 😜'))

print(ord('™'))
print(ord('😜'))

True
True
8482
128540


### Part two

To minimize the impact of data loss, we'll only remove an app if its name has more than three non-ASCII characters:

In [15]:
def is_english(string):
    non_ascii = 0
    
    for character in string:
        if ord(character) > 127:
            non_ascii += 1
    if non_ascii > 3:
        return False
    else:
        return True

The function is still not perfect, and very few non-English apps might get past our filter, but this seems good enough at this point in my analysis — I shouldn't spend too much time on optimization at this point.

Below, I use the is_english() function to filter out the non-English apps for both data sets:

In [16]:
android_english = []
ios_english = []

for app in android_clean:
    name = app[0]
    if is_english(name):
        android_english.append(app)
        
for app in ios:
    name = app[0]
    if is_english(name):
        ios_english.append(app)
        

Can see that we're left with 9614 Android apps and 6183 iOS apps.

## Isolating the Free Apps

As mentioned in the introduction. I only build apps that are free to download and install, and our main source of revenue consists of in-app ads. Our data sets contain both free and non-free apps, and we'll need to isolate only the free apps for our analysis. Below, we isolate the free apps for both our data sets.

In [17]:
android_final = []
ios_final = []

for app in android_english:
    price = app[7]
    if price == '0':
        android_final.append(app)
for app in ios_english:
    price = app[4]
    if price == '0.0':
        ios_final.append(app)

print(len(android_final))
print(len(ios_final))
        

9226
4056


## Most Common Apps by Genre



### Part one

As I mentioned in the introduction, my aim is to determine the kinds of apps that are likely to attract more users because the revenues is highly influenced by the number of people using apps.

To minimize risks and overhead, my validation strategy for an app idea is comprised of three steps:
1. Build a minimal Android version of the app, and add it to Google Play.
2. If the app has a good response from users, then develop it further 
3. If the app is profitable after six months, we also build an iOS version of the app and add it to the App Store

Because the end goal is to add the app on both thr App Store and Google Play, I need to find app profiles that are succesful on both markets. For instance, a profile that might work well both markets might be a productivity app that makes use of gmamification.

Let's begin the analysis by getting a sense of the most common genres for each market. For this, build a frequency table for the prime_genre column of the App Store data set, and the Genres and Category columns of the Google Play data set.

### Part two

I'll build two functions I can use to analyze the frequency tables:
- One function to generate frequency tables that show percentages.
- Another function that I can use to display the percentages in a descending order.

In [18]:
def freq_table(dataset, index):
    table = {}
    total = 0
    
    for row in dataset:
        total += 1
        value = row[index]
        if value in table:
            table[value] += 1
        else:
            table[value] = 1
        
    table_percentage = {}
    for key in table:
        percentage = (table[key]/total)*100
        table_percentage[key] = percentage
    return table_percentage
        
def display_table(dataset, index):
    table = freq_table(dataset, index)
    table_display = []
    
    for key in table:
        key_val_as_tuple = (table[key], key)
        table_display.append(key_val_as_tuple)
    
    table_sorted = sorted(table_display, reverse = True)
    for entry in table_sorted:
        print(entry[1], ':', entry[0])

### Part three

We start by examining the frequency table for the prime_genre column of the App Store data set.

In [19]:
display_table(ios_final, -5)

Games : 55.64595660749507
Entertainment : 8.234714003944774
Photo & Video : 4.117357001972387
Social Networking : 3.5256410256410255
Education : 3.2544378698224854
Shopping : 2.983234714003945
Utilities : 2.687376725838264
Lifestyle : 2.3175542406311638
Finance : 2.0710059171597637
Sports : 1.947731755424063
Health & Fitness : 1.8737672583826428
Music : 1.6518737672583828
Book : 1.6272189349112427
Productivity : 1.5285996055226825
News : 1.4299802761341223
Travel : 1.3806706114398422
Food & Drink : 1.0601577909270217
Weather : 0.7642998027613412
Reference : 0.4930966469428008
Navigation : 0.4930966469428008
Business : 0.4930966469428008
Catalogs : 0.22189349112426035
Medical : 0.19723865877712032


We can see that among the free English apps, more than a half(55.65%) are games. Entertainment apps are close to 8% followed by phto and video apps, which are close to 4%. Only 3.53 of the apps are designed for social and networking apps, followed by education apps which amount for 3.23% of the apps in our data set.

The general impression is that App Store(at least the part containing free English apps) is do,incated by apps that are designed for fun(games, entertainment, photo and video, social networking, sports, music, etc). While apps with practical purposes (education, shopping, utilies, productivity, lifestyle, etc) are more rare. However, the fact that fun apps are the most numerous doesn's also imply that they also have the greatest number of users - the demand might not be the same as the offer.

Let's continue by examining the Genres and Category columns of the Google Play data set (two columns which seem to be related).

In [20]:
display_table(android_final, 1) # Category

FAMILY : 18.42618686321266
GAME : 9.473227834381097
TOOLS : 8.140039020160417
BUSINESS : 4.7257749837416
PRODUCTIVITY : 3.891177108172556
LIFESTYLE : 3.8478213743767613
MEDICAL : 3.7177541729893777
FINANCE : 3.598525905050943
SPORTS : 3.479297637112508
HEALTH_AND_FITNESS : 3.295035768480382
COMMUNICATION : 3.2408411012356386
PERSONALIZATION : 3.2191632343377417
PHOTOGRAPHY : 2.904834164318231
NEWS_AND_MAGAZINES : 2.8723173639713853
SOCIAL : 2.6988944287882073
SHOPPING : 2.5146325601560804
TRAVEL_AND_LOCAL : 2.4279210925644916
DATING : 2.12443095599393
BOOKS_AND_REFERENCE : 2.070236288749187
VIDEO_PLAYERS : 1.7233904183828312
MAPS_AND_NAVIGATION : 1.3440277476696292
EDUCATION : 1.322349880771732
FOOD_AND_DRINK : 1.2247994797311943
ENTERTAINMENT : 1.1814437459354
LIBRARIES_AND_DEMO : 0.8996314762627358
AUTO_AND_VEHICLES : 0.8887925428137872
HOUSE_AND_HOME : 0.8779536093648386
WEATHER : 0.7695642748753523
EVENTS : 0.6828528072837633
PARENTING : 0.6286581400390201
ART_AND_DESIGN : 0.617819

The landscape seems significantly different on Google Play: there are not that many apps designed for fun, and it seems that a good number of apps are designed for practical purposes (family, tools, business, lifestyle, productivity, etc.). However, if we investigate this further, we can see that the family category (which accounts for almost 19% of the apps) means mostly games for kids.

Even so, practical apps seem to have a better representation on Google Play compared to App Store. This picture is also confirmed by the frequency table we see for the Genres column:

In [21]:
display_table(android_final, -4)

Tools : 8.129200086711467
Entertainment : 6.1023195317580745
Education : 5.332755256882723
Business : 4.7257749837416
Productivity : 3.891177108172556
Lifestyle : 3.836982440927813
Medical : 3.7177541729893777
Finance : 3.598525905050943
Sports : 3.5551701712551487
Health & Fitness : 3.295035768480382
Communication : 3.2408411012356386
Personalization : 3.2191632343377417
Action : 3.0132234988077173
Photography : 2.904834164318231
News & Magazines : 2.8723173639713853
Social : 2.6988944287882073
Shopping : 2.5146325601560804
Travel & Local : 2.4170821591155427
Dating : 2.12443095599393
Books & Reference : 2.070236288749187
Simulation : 1.9726858877086493
Arcade : 1.7884240190765228
Video Players & Editors : 1.7017125514849338
Casual : 1.7017125514849338
Maps & Navigation : 1.3440277476696292
Food & Drink : 1.2247994797311943
Puzzle : 1.1164101452417081
Racing : 0.9538261435074789
Role Playing : 0.8996314762627358
Libraries & Demo : 0.8996314762627358
Strategy : 0.8887925428137872
Auto 

The difference between the Genres and the Category columns is not crystal clear, but one thing we can notice is that the Genres column is much more granular (it has more categories). We're only looking for the bigger picture at the moment, so we'll only work with the Category column moving forward.

Up to this point, we found that the App Store is dominated by apps designed for fun, while Google Play shows a more balanced landscape of both practical and for-fun apps. Now we'd like to get an idea about the kind of apps that have most users.

## Most Popular Apps by Genre on the App Store

One way to find out what genres are the most popular (have the most users) is to calculate the average number of installs for each app genre. For the Google Play data set, we can find this information in the Installs column, but this information is missing for the App Store data set. As a workaround, we'll take the total number of user ratings as a proxy, which we can find in the rating_count_tot app.

Let's start with calculating the average number of user ratings per app genre on the App Store. To do that, we'll need to do the following:

- Isolate the apps of each genre
- Add up the user ratings for the apps of that genre
- Divide the sum by the number of apps belonging to that genre (not by the total number of apps)

To calculate the average number of user ratings for each genre, we'll use a for loop inside of another for loop. This is an example of a for loop used inside another for loop: